In [1]:
# Import required Dependencies
import os
import sys
import numpy as np
#import matplotlib.pyplot as plt
import pandas as pd
#import seaborn as sns
import sklearn
import datetime

# Pre Processing
from sklearn.preprocessing import MinMaxScaler
#from sklearn.preprocessing import LabelEncoder

# Classifiers
#from sklearn import svm
#from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

# DL models
#import tensorflow as tf
#from tensorflow.keras import layers 
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Input
#from tensorflow.keras.models import Model
#from tensorflow.python.keras.layers import deserialize, serialize
#from tensorflow.python.keras.saving import saving_utils

# Save models
import pickle

#Geotiff
import rasterio

In [2]:
import os

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "CC"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution, bbox_inches='tight')

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [2]:
# Load the saved data
df = pd.read_csv('CC_all_20230727.csv')

In [4]:
df.head()

,awei,blue,dbsi,evi,green,kndvi,mndmi,ndbi,ndpi,ndvi,ndwi,nir,osavi,red,savi,str11,str12,VH,VV-VH,VV
0,962.25,409.0,-0.365920,0.059429,325.0,0.058976,0.608911,-0.012500,-0.608911,-0.242991,0.600985,81.0,-12918.1312,133.0,-16731.00,38.506329,37.506494,0.000288,7.414589,0.002133
1,891.75,401.0,-0.317651,0.062776,297.0,0.068607,0.579787,0.019355,-0.579787,-0.262136,0.592493,76.0,-12913.8624,130.0,-16726.50,38.506329,37.506494,0.000130,17.670248,0.002293
2,959.25,409.0,-0.430322,0.046574,317.0,0.043204,0.638243,-0.066667,-0.638243,-0.207921,0.596977,80.0,-9849.2352,122.0,-12757.50,34.007143,33.507246,0.000016,78.076813,0.001272
3,1000.25,430.0,-0.429883,0.045833,325.0,0.046487,0.645570,-0.066667,-0.645570,-0.215686,0.604938,80.0,-10420.3264,124.0,-13497.00,34.007143,33.507246,0.000032,19.939922,0.000639
4,990.50,436.0,-0.446256,0.037724,325.0,0.031939,0.625000,-0.062500,-0.625000,-0.178744,0.585366,85.0,-8891.3072,122.0,-11516.25,36.506667,35.006944,0.000018,155.772690,0.002747


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120560400 entries, 0 to 120560399
Data columns (total 20 columns):
 #   Column  Dtype  
---  ------  -----  
 0   awei    float64
 1   blue    float64
 2   dbsi    float64
 3   evi     float64
 4   green   float64
 5   kndvi   float64
 6   mndmi   float64
 7   ndbi    float64
 8   ndpi    float64
 9   ndvi    float64
 10  ndwi    float64
 11  nir     float64
 12  osavi   float64
 13  red     float64
 14  savi    float64
 15  str11   float64
 16  str12   float64
 17  VH      float64
 18  VV-VH   float64
 19  VV      float64
dtypes: float64(20)
memory usage: 18.0 GB


In [6]:
df.describe()

,awei,blue,dbsi,evi,green,kndvi,mndmi,ndbi,ndpi,ndvi,ndwi,nir,osavi,red,savi,str11,str12,VH,VV-VH,VV
count,1.205604e+08,1.205604e+08,1.205526e+08,1.205604e+08,1.205604e+08,1.205526e+08,1.205599e+08,1.205599e+08,1.205599e+08,1.205526e+08,1.205594e+08,1.205604e+08,1.205604e+08,1.205604e+08,1.205604e+08,1.205604e+08,1.205604e+08,1.205604e+08,1.205604e+08,1.205604e+08
mean,-4.462877e+03,5.428684e+02,-2.210391e-01,-1.576497e+00,7.553603e+02,3.640226e-01,-2.314373e-01,-2.279930e-01,2.314373e-01,4.524693e-01,-3.725659e-01,2.758638e+03,1.017849e+07,6.854185e+02,1.316289e+07,8.236723e+02,5.634792e+02,1.686121e-02,2.518611e+01,9.104504e-02
std,2.991744e+03,4.098897e+02,2.293991e-01,1.832855e+03,4.943895e+02,2.490563e-01,4.546110e-01,2.389484e-01,4.546110e-01,4.571331e-01,5.166087e-01,1.463356e+03,8.387827e+06,7.123683e+02,1.084709e+07,5.062268e+02,4.798425e+02,4.220213e-02,1.132117e+02,7.155829e-01
min,-3.521700e+04,0.000000e+00,-1.795745e+00,-3.300000e+05,0.000000e+00,0.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,-4.570903e+07,0.000000e+00,-5.910854e+07,0.000000e+00,0.000000e+00,1.644899e-07,6.770266e-04,9.349317e-07
25%,-6.585500e+03,3.080000e+02,-3.699685e-01,4.027149e-01,4.360000e+02,9.751995e-02,-4.959538e-01,-4.204993e-01,2.689982e-01,1.186703e-01,-7.393768e-01,2.210000e+03,2.469097e+06,2.400000e+02,3.193038e+06,6.405004e+02,3.205008e+02,5.047485e-03,2.996706e+00,2.341818e-02
50%,-5.382500e+03,3.940000e+02,-2.988851e-01,1.979400e+00,6.060000e+02,4.200827e-01,-4.198773e-01,-2.761240e-01,4.198773e-01,6.567817e-01,-6.049896e-01,3.124000e+03,9.880042e+06,4.100000e+02,1.277704e+07,8.040003e+02,4.400006e+02,1.307114e-02,5.158920e+00,6.121234e-02
75%,-3.175250e+03,5.780000e+02,-1.045573e-01,2.607357e+00,8.500000e+02,6.038794e-01,-2.689982e-01,-5.230957e-02,4.959538e-01,8.344371e-01,-2.162417e-01,3.764000e+03,1.600539e+07,7.970000e+02,2.069833e+07,1.028000e+03,6.500004e+02,2.353029e-02,9.832112e+00,1.141939e-01
max,1.587900e+04,8.888000e+03,2.000000e+00,3.300000e+05,9.192000e+03,7.615942e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.282400e+04,1.628266e+08,9.480000e+03,2.105557e+08,6.967000e+03,7.595500e+03,1.523495e+02,3.100643e+05,3.000135e+03


In [3]:
df = df.astype('float32')

In [8]:
df.head()

,awei,blue,dbsi,evi,green,kndvi,mndmi,ndbi,ndpi,ndvi,ndwi,nir,osavi,red,savi,str11,str12,VH,VV-VH,VV
0,962.25,409.0,-0.365920,0.059429,325.0,0.058976,0.608911,-0.012500,-0.608911,-0.242991,0.600985,81.0,-12918.130859,133.0,-16731.00,38.506329,37.506493,0.000288,7.414589,0.002133
1,891.75,401.0,-0.317651,0.062776,297.0,0.068607,0.579787,0.019355,-0.579787,-0.262136,0.592493,76.0,-12913.862305,130.0,-16726.50,38.506329,37.506493,0.000130,17.670248,0.002293
2,959.25,409.0,-0.430322,0.046574,317.0,0.043204,0.638243,-0.066667,-0.638243,-0.207921,0.596977,80.0,-9849.235352,122.0,-12757.50,34.007141,33.507248,0.000016,78.076813,0.001272
3,1000.25,430.0,-0.429883,0.045833,325.0,0.046487,0.645570,-0.066667,-0.645570,-0.215686,0.604938,80.0,-10420.326172,124.0,-13497.00,34.007141,33.507248,0.000032,19.939922,0.000639
4,990.50,436.0,-0.446256,0.037724,325.0,0.031939,0.625000,-0.062500,-0.625000,-0.178744,0.585366,85.0,-8891.307617,122.0,-11516.25,36.506668,35.006943,0.000018,155.772690,0.002747


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120560400 entries, 0 to 120560399
Data columns (total 20 columns):
 #   Column  Dtype  
---  ------  -----  
 0   awei    float32
 1   blue    float32
 2   dbsi    float32
 3   evi     float32
 4   green   float32
 5   kndvi   float32
 6   mndmi   float32
 7   ndbi    float32
 8   ndpi    float32
 9   ndvi    float32
 10  ndwi    float32
 11  nir     float32
 12  osavi   float32
 13  red     float32
 14  savi    float32
 15  str11   float32
 16  str12   float32
 17  VH      float32
 18  VV-VH   float32
 19  VV      float32
dtypes: float32(20)
memory usage: 9.0 GB


In [10]:
df.describe()

,awei,blue,dbsi,evi,green,kndvi,mndmi,ndbi,ndpi,ndvi,ndwi,nir,osavi,red,savi,str11,str12,VH,VV-VH,VV
count,1.205604e+08,1.205604e+08,1.205526e+08,1.205604e+08,1.205604e+08,1.205526e+08,1.205599e+08,1.205599e+08,1.205599e+08,1.205526e+08,1.205594e+08,1.205604e+08,1.205604e+08,1.205604e+08,120560400.0,1.205604e+08,1.205604e+08,1.205604e+08,1.205604e+08,1.205604e+08
mean,-4.462875e+03,5.428680e+02,-2.210384e-01,-1.576493e+00,7.553598e+02,3.640226e-01,-2.314374e-01,-2.279930e-01,2.314374e-01,4.524686e-01,-3.725668e-01,2.758642e+03,1.017850e+07,6.854184e+02,13162852.0,8.236735e+02,5.634774e+02,1.686124e-02,2.518618e+01,9.104492e-02
std,3.081942e+03,4.469239e+02,1.846113e-01,1.832828e+03,5.778926e+02,2.637890e-01,3.687679e-01,1.881849e-01,3.687679e-01,3.733466e-01,3.731667e-01,1.560464e+03,8.088852e+06,7.222431e+02,10552574.0,5.735591e+02,5.128111e+02,4.054161e-02,1.119266e+02,7.103187e-01
min,-3.521700e+04,0.000000e+00,-1.795745e+00,-3.300000e+05,0.000000e+00,0.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,-4.570902e+07,0.000000e+00,-59108540.0,0.000000e+00,0.000000e+00,1.644899e-07,6.770266e-04,9.349317e-07
25%,-6.585500e+03,3.080000e+02,-3.699685e-01,4.027149e-01,4.360000e+02,9.751995e-02,-4.959538e-01,-4.204994e-01,2.689981e-01,1.186703e-01,-7.393768e-01,2.210000e+03,2.469097e+06,2.400000e+02,3193038.0,6.405004e+02,3.205008e+02,5.047485e-03,2.996706e+00,2.341818e-02
50%,-5.382500e+03,3.940000e+02,-2.988851e-01,1.979400e+00,6.060000e+02,4.200827e-01,-4.198773e-01,-2.761240e-01,4.198773e-01,6.567817e-01,-6.049896e-01,3.124000e+03,9.880042e+06,4.100000e+02,12777041.0,8.040003e+02,4.400006e+02,1.307114e-02,5.158920e+00,6.121234e-02
75%,-3.175250e+03,5.780000e+02,-1.045573e-01,2.607357e+00,8.500000e+02,6.038793e-01,-2.689981e-01,-5.230958e-02,4.959538e-01,8.344371e-01,-2.162417e-01,3.764000e+03,1.600539e+07,7.970000e+02,20698330.0,1.028000e+03,6.500004e+02,2.353029e-02,9.832112e+00,1.141939e-01
max,1.587900e+04,8.888000e+03,2.000000e+00,3.300000e+05,9.192000e+03,7.615942e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.282400e+04,1.628266e+08,9.480000e+03,210555712.0,6.967000e+03,7.595500e+03,1.523495e+02,3.100643e+05,3.000135e+03


In [11]:
np.isinf(df).values.sum() 

0

In [12]:
df.isnull().sum()

awei        0
blue        0
dbsi     7819
evi         6
green       0
kndvi    7819
mndmi     512
ndbi      518
ndpi      512
ndvi     7819
ndwi     1035
nir         0
osavi       0
red         0
savi        0
str11       0
str12       0
VH          0
VV-VH       0
VV          0
dtype: int64

In [5]:
# Replacing the nan to zero
df.replace(np.nan, 0, inplace=True)

# Data Preparation for Model Building

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
# Creating training and test sets
# Splitting the data into train and test
X = df.iloc[:, :]

# Load the scaler object
with open('models/CC/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

# Transform the 2023 data using the loaded scaler
X_scaled = scaler.transform(X)

print(X_scaled.shape)

(120560400, 24)


In [8]:
X_scaled.dtype

dtype('float32')

In [6]:
import gc
del df
del scaler
gc.collect()

0

## Classifiers

In [14]:
# Copy the profile of sentinel-2 band
with rasterio.open("LU/2023-08-06/blue_2023-08-06.tif") as src:
    out_profile = src.profile.copy()

In [11]:
#LU = df.class.unique()
CC = ['Maize', 'Rice', 'Other']
CC

['Cultivated_area',
 'Trees_palms',
 'Buildings',
 'Roads',
 'Water_bodies',
 'Aqua_culture']

### KNeighborsClassifier

In [9]:
# Load saved model
pickled_knn_model = pickle.load(open('models/CC/knn_cv.pkl', 'rb'))

In [ ]:
print(datetime.datetime.now())
y_pred_knn = pickled_knn_model.predict(X_scaled)
print(datetime.datetime.now())

2024-06-11 18:47:38.904606


In [9]:
y_pred_knn

array([1., 2., 2., ..., 1., 1., 1.])

In [10]:
# reshape to 2d
y_pred_knn_2d = np.reshape(y_pred_knn,(10980,10980))

In [11]:
y_pred_knn_2d

array([[1., 2., 2., ..., 1., 1., 3.],
       [3., 2., 2., ..., 1., 1., 2.],
       [1., 2., 2., ..., 2., 3., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 2., ..., 1., 1., 2.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [13]:
# Save the prediction result as a geotif file
with rasterio.open("images/CC/predict/knn_20230622.tif", 'w', **out_profile) as dst:
    dst.write(y_pred_knn_2d, indexes=1)

### DecisionTreeClassifier

In [17]:
# Load saved model
pickled_dt_model = pickle.load(open('models/CC/dt_cv.pkl', 'rb'))

In [18]:
print(datetime.datetime.now())
y_pred_dt = pickled_dt_model.predict(X_scaled)
print(datetime.datetime.now())

2024-06-26 23:17:47.183300
2024-06-26 23:18:08.634896


In [19]:
y_pred_dt

array([1., 1., 1., ..., 1., 1., 1.])

In [20]:
# reshape to 2d
y_pred_dt_2d = np.reshape(y_pred_dt,(10980,10980))

In [21]:
y_pred_dt_2d

array([[1., 1., 1., ..., 3., 3., 3.],
       [1., 3., 3., ..., 3., 3., 3.],
       [3., 1., 1., ..., 3., 3., 3.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [22]:
# Save the prediction result as a geotif file
with rasterio.open("images/CC/predict/dt_20230727.tif", 'w', **out_profile) as dst:
    dst.write(y_pred_dt_2d, indexes=1)

In [23]:
del pickled_dt_model
del y_pred_dt
del y_pred_dt_2d

### RandomForestClassifier

In [24]:
# Load saved model 
pickled_rf_cv = pickle.load(open('models/CC/rf_cv.pkl', 'rb'))

In [25]:
print(datetime.datetime.now())
y_pred_rf = pickled_rf_cv.predict(X_scaled)
print(datetime.datetime.now())

2024-06-26 23:18:19.417569
2024-06-27 02:19:43.648285


In [26]:
y_pred_rf

array([3., 3., 3., ..., 1., 1., 1.])

In [27]:
# reshape to 2d
y_pred_rf_2d = np.reshape(y_pred_rf,(10980,10980))

In [28]:
y_pred_rf_2d

array([[3., 3., 3., ..., 3., 3., 1.],
       [3., 3., 3., ..., 3., 3., 3.],
       [3., 3., 3., ..., 3., 3., 3.],
       ...,
       [1., 1., 3., ..., 3., 1., 1.],
       [1., 1., 3., ..., 1., 1., 1.],
       [1., 3., 3., ..., 1., 1., 1.]])

In [29]:
# Save the prediction result as a geotif file
with rasterio.open("images/CC/predict/rf_20230727.tif", 'w', **out_profile) as dst:
    dst.write(y_pred_rf_2d, indexes=1)

In [30]:
del pickled_rf_cv

### XGBoost (extreme Gradient Boosting) Classifier

In [7]:
# Load saved model 
pickled_xgb_cv = pickle.load(open('models/CC/xgb_cv.pkl', 'rb'))

In [8]:
print(datetime.datetime.now())
y_pred = pickled_xgb_cv.predict(X_scaled)
print(datetime.datetime.now())

2024-06-26 23:13:19.155095
2024-06-26 23:17:14.128185


In [9]:
y_pred

array([2, 2, 2, ..., 0, 0, 0], dtype=int64)

In [10]:
# reshape to 2d
y_pred_2d = np.reshape(y_pred,(10980,10980))

In [11]:
y_pred_2d

array([[2, 2, 2, ..., 2, 2, 2],
       [2, 2, 2, ..., 2, 2, 2],
       [2, 2, 0, ..., 2, 2, 2],
       ...,
       [0, 0, 2, ..., 2, 0, 0],
       [0, 0, 2, ..., 0, 0, 0],
       [0, 2, 2, ..., 0, 0, 0]], dtype=int64)

In [12]:
y_pred_2d_mod = y_pred_2d + 1

In [13]:
y_pred_2d_mod

array([[3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3],
       [3, 3, 1, ..., 3, 3, 3],
       ...,
       [1, 1, 3, ..., 3, 1, 1],
       [1, 1, 3, ..., 1, 1, 1],
       [1, 3, 3, ..., 1, 1, 1]], dtype=int64)

In [15]:
# Save the prediction result as a geotif file
with rasterio.open("images/CC/predict/xgb_20230727.tif", 'w', **out_profile) as dst:
    dst.write(y_pred_2d_mod, indexes=1)

In [16]:
del pickled_xgb_cv
del y_pred
del y_pred_2d
del y_pred_2d_mod

### SVM RBF

In [17]:
import os
os.add_dll_directory("C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v12.3\\bin")

<AddedDllDirectory('C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v12.3\\bin')>

In [18]:
import thundersvm
from thundersvm import SVC

In [19]:
# Load saved model 
pickled_svc_cv = pickle.load(open('models/CC/svc_rbf_cv.pkl', 'rb'))

In [ ]:
# Evaluate the model on the entire dataset
print(datetime.datetime.now())
y_pred_svc = pickled_svc_cv.predict(X_scaled)
print(datetime.datetime.now())

2024-06-08 19:15:03.609216


In [11]:
y_pred_svc

OSError: [WinError -529697949] Windows Error 0xe06d7363

In [ ]:
# reshape to 2d
y_pred_svc_2d = np.reshape(y_pred_svc,(10980,10980))

In [ ]:
# Save the prediction result as a geotif file
with rasterio.open("images/CC/predict/svc_20230622.tif", 'w', **out_profile) as dst:
    dst.write(y_pred_svc_2d, indexes=1)

### LSTM

In [13]:
def unpack(model, training_config, weights):
    restored_model = deserialize(model)
    if training_config is not None:
        restored_model.compile(
            **saving_utils.compile_args_from_training_config(
                training_config
            )
        )
    restored_model.set_weights(weights)
    return restored_model

# Hotfix function
def make_keras_picklable():

    def __reduce__(self):
        model_metadata = saving_utils.model_metadata(self)
        training_config = model_metadata.get("training_config", None)
        model = serialize(self)
        weights = self.get_weights()
        return (unpack, (model, training_config, weights))

    cls = Model
    cls.__reduce__ = __reduce__

# Run the function
make_keras_picklable()

In [14]:
#Load the saved model
pickled_lstm_model = pickle.load(open('models/CC/lstm_model.pkl', 'rb'))

In [15]:
# Reshape the X shape to fit in LSTM (add one dimension)
X_scaled_reshaped = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))

In [16]:
print(X_scaled_reshaped.shape)

(120560400, 20, 1)


In [ ]:
print(datetime.datetime.now())
y_pred_lstm = np.argmax(pickled_lstm_model.predict(X_scaled_reshaped), axis=-1)
print(datetime.datetime.now())

2024-06-07 14:56:50.004398


In [18]:
y_pred_lstm

2024-03-03 10:06:55.018031
Epoch 1/100
3727/3727 [==============================] - 106791s 29s/step - loss: 0.7491 - accuracy: 0.7073 - val_loss: 0.6010 - val_accuracy: 0.7674
Epoch 2/100
3727/3727 [==============================] - 16609s 4s/step - loss: 0.5573 - accuracy: 0.7914 - val_loss: 0.4565 - val_accuracy: 0.8323
Epoch 3/100
3727/3727 [==============================] - 16357s 4s/step - loss: 0.4852 - accuracy: 0.8227 - val_loss: 0.4514 - val_accuracy: 0.8332
Epoch 4/100
3727/3727 [==============================] - 16201s 4s/step - loss: 0.4668 - accuracy: 0.8298 - val_loss: 0.4327 - val_accuracy: 0.8411
Epoch 5/100
3727/3727 [==============================] - 18723s 5s/step - loss: 0.4568 - accuracy: 0.8336 - val_loss: 0.4241 - val_accuracy: 0.8440
Epoch 6/100
3727/3727 [==============================] - 36917s 10s/step - loss: 0.4491 - accuracy: 0.8366 - val_loss: 0.4180 - val_accuracy: 0.8463
Epoch 7/100
3727/3727 [==============================] - 16238s 4s/step - loss: 0.

In [19]:
# reshape to 2d
y_pred_lstm_2d = np.reshape(y_pred,(10980,10980))

In [20]:
y_pred_lstm_2d

In [21]:
y_pred_lstm_2d_mod = y_pred_lstm_2d + 1

array([0, 0, 5, ..., 4, 1, 4], dtype=int64)

In [22]:
# Save the prediction result as a geotif file
with rasterio.open("images/CC/predict/lstm_20230622.tif", 'w', **out_profile) as dst:
    dst.write(y_pred_lstm_2d_mod, indexes=1)

array([[1193454,  131725,    4818,   47830,   21307,   13322],
       [  42693, 1347161,    2656,   15159,    5634,     182],
       [   2846,    2945, 1327123,   79523,    1021,     702],
       [  43177,   19090,  118133, 1199859,   11193,   22644],
       [  65189,   25755,    1335,   49299, 1211260,   60600],
       [  21331,     572,     430,   60481,   89644, 1239272]],
      dtype=int64)